In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats
from sklearn.model_selection import train_test_split


# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

#My imports
import wrangle as w
import env as e
import os

In [2]:
pd.options.display.float_format = '{:,.3f}'.format

Exercises
Using the repo setup directions, setup a new local and remote repository named clustering-exercises s. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named clustering-exercises

Save your clustering work in your clustering-exercises repo. Then add, commit, and push your changes.

For example, if the exercise directs you to create a file named myfile.py, you should have clustering/myfile.py in your repository.

If a file extension is specified, you should create that specific file. If there is not file extension specified, you may either create a python script or a jupyter notebook for the exercise.

### Zillow
For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

### Acquire and Summarize

1. Acquire data from the cloud database. You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database. Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [23]:
# SQL query
query = '''SELECT *
FROM properties_2017
LEFT JOIN predictions_2017 USING (parcelid)
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
LEFT JOIN buildingclasstype USING (buildingclasstypeid)
LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
LEFT JOIN airconditioningtype USING (airconditioningtypeid)
LEFT JOIN storytype USING (storytypeid)
LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
LEFT JOIN propertylandusetype USING (propertylandusetypeid)
LEFT JOIN unique_properties USING (parcelid)
WHERE transactiondate LIKE '2017%%'
AND latitude is not NULL
AND longitude is not NULL
AND (propertylandusetypeid = 261 OR propertylandusetypeid = 279);'''

In [24]:
def acquire_zillow():
    ''' Retrieve data from Zillow database within codeup, selecting specific features
    If data is not present in directory, this function writes a copy as a csv file. 
    '''
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        query = '''SELECT *
        FROM properties_2017
        LEFT JOIN predictions_2017 USING (parcelid)
        LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
        LEFT JOIN buildingclasstype USING (buildingclasstypeid)
        LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
        LEFT JOIN airconditioningtype USING (airconditioningtypeid)
        LEFT JOIN storytype USING (storytypeid)
        LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
        LEFT JOIN propertylandusetype USING (propertylandusetypeid)
        LEFT JOIN unique_properties USING (parcelid)
        WHERE transactiondate LIKE '2017%%'
        AND latitude is not NULL
        AND longitude is not NULL
        AND (propertylandusetypeid = 261 OR propertylandusetypeid = 279);'''
        df = pd.read_sql(query, w.get_connection('zillow'))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename, index=False)

        # Return the dataframe to the calling code
        return df

In [3]:
df = w.acquire_zillow()

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [26]:
df.head()

,parcelid,propertylandusetypeid,typeconstructiontypeid,storytypeid,airconditioningtypeid,architecturalstyletypeid,buildingclasstypeid,heatingorsystemtypeid,id,basementsqft,...,id.1,logerror,transactiondate,heatingorsystemdesc,buildingclassdesc,architecturalstyledesc,airconditioningdesc,storydesc,typeconstructiondesc,propertylandusedesc
0,14297519,261.000,NaN,NaN,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.026,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
1,17052889,261.000,NaN,NaN,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.056,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
2,14186244,261.000,NaN,NaN,NaN,NaN,NaN,NaN,11677,NaN,...,2,0.005,2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
3,12177905,261.000,NaN,NaN,NaN,NaN,NaN,2.000,2288172,NaN,...,3,-0.103,2017-01-01,Central,NaN,NaN,NaN,NaN,NaN,Single Family Residential
4,12095076,261.000,NaN,NaN,1.000,NaN,NaN,2.000,781532,NaN,...,6,-0.001,2017-01-01,Central,NaN,NaN,Central,NaN,NaN,Single Family Residential


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   propertylandusetypeid         52441 non-null  float64
 2   typeconstructiontypeid        76 non-null     float64
 3   storytypeid                   47 non-null     float64
 4   airconditioningtypeid         13638 non-null  float64
 5   architecturalstyletypeid      70 non-null     float64
 6   buildingclasstypeid           0 non-null      float64
 7   heatingorsystemtypeid         33935 non-null  float64
 8   id                            52441 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52441 non-null  float64
 11  bedroomcnt                    52441 non-null  float64
 12  buildingqualitytypeid         33740 non-null  float64
 13  c

In [28]:
print(f'rows: {df.shape[0]}')
print(f'co1umns: {df.shape[1]}')

rows: 52441
co1umns: 69


In [8]:
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print('\n-------------------------- \n')

parcelid
11991059    3
11957553    2
12478591    2
12035592    2
12443331    2
           ..
11494547    1
10976762    1
12650619    1
12650850    1
12826780    1
Name: parcelid, Length: 52320, dtype: int64

-------------------------- 

propertylandusetypeid
261.00    52441
Name: propertylandusetypeid, dtype: int64

-------------------------- 

typeconstructiontypeid
6.00    75
4.00     1
Name: typeconstructiontypeid, dtype: int64

-------------------------- 

storytypeid
7.00    47
Name: storytypeid, dtype: int64

-------------------------- 

airconditioningtypeid
 1.00    11895
13.00     1568
 5.00      159
11.00       16
Name: airconditioningtypeid, dtype: int64

-------------------------- 

architecturalstyletypeid
 7.00    62
 3.00     3
 2.00     2
21.00     2
 8.00     1
Name: architecturalstyletypeid, dtype: int64

-------------------------- 

buildingclasstypeid
Series([], Name: buildingclasstypeid, dtype: int64)

-------------------------- 

heatingorsystemtypeid
 2.00    207

In [9]:
for column in df.columns:
    print(f'{column}\n{df[column].value_counts()} \n\n ------------------------- \n')

parcelid
11991059    3
11957553    2
12478591    2
12035592    2
12443331    2
           ..
11494547    1
10976762    1
12650619    1
12650850    1
12826780    1
Name: parcelid, Length: 52320, dtype: int64 

 ------------------------- 

propertylandusetypeid
261.00    52441
Name: propertylandusetypeid, dtype: int64 

 ------------------------- 

typeconstructiontypeid
6.00    75
4.00     1
Name: typeconstructiontypeid, dtype: int64 

 ------------------------- 

storytypeid
7.00    47
Name: storytypeid, dtype: int64 

 ------------------------- 

airconditioningtypeid
 1.00    11895
13.00     1568
 5.00      159
11.00       16
Name: airconditioningtypeid, dtype: int64 

 ------------------------- 

architecturalstyletypeid
 7.00    62
 3.00     3
 2.00     2
21.00     2
 8.00     1
Name: architecturalstyletypeid, dtype: int64 

 ------------------------- 

buildingclasstypeid
Series([], Name: buildingclasstypeid, dtype: int64) 

 ------------------------- 

heatingorsystemtypeid
 2.00

propertyzoningdesc
LAR1          6498
LARS          1385
LBR1N         1111
SCUR2          712
LARE11         612
              ... 
LCR3800015       1
PSR6*            1
LARE             1
MYR3PY           1
BFA15000*        1
Name: propertyzoningdesc, Length: 1286, dtype: int64 

 ------------------------- 

rawcensustractandblock
60,379,203.39    36
60,379,201.16    32
60,371,417.00    24
60,590,320.57    24
60,590,320.42    23
                 ..
60,375,775.01     1
60,375,775.04     1
60,375,353.00     1
60,372,407.01     1
60,375,034.02     1
Name: rawcensustractandblock, Length: 31440, dtype: int64 

 ------------------------- 

regionidcity
12,447.00    11452
 5,534.00     1795
40,227.00     1492
46,298.00     1428
16,764.00     1087
             ...  
32,927.00        3
31,134.00        2
36,078.00        1
10,815.00        1
21,395.00        1
Name: regionidcity, Length: 175, dtype: int64 

 ------------------------- 

regionidcounty
3,101.00    33910
1,286.00    14136
2,061.

In [10]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [11]:
w.summarize(df)

SUMMARY REPORT


Dataframe head: 
   parcelid  propertylandusetypeid  typeconstructiontypeid  storytypeid  \
0  14297519                 261.00                     NaN          NaN   
1  17052889                 261.00                     NaN          NaN   
2  14186244                 261.00                     NaN          NaN   

   airconditioningtypeid  architecturalstyletypeid  buildingclasstypeid  \
0                    NaN                       NaN                  NaN   
1                    NaN                       NaN                  NaN   
2                    NaN                       NaN                  NaN   

   heatingorsystemtypeid       id  basementsqft  ...  id.1  \
0                    NaN  1727539           NaN  ...     0   
1                    NaN  1387261           NaN  ...     1   
2                    NaN    11677           NaN  ...     2   

              logerror  transactiondate  heatingorsystemdesc  \
0                 0.03       2017-01-01            

ValueError: Bin edges must be unique: array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]).
You can drop duplicate edges by setting the 'duplicates' kwarg

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

### Nulls by Column

In [ ]:
df.isnull().sum()

In [12]:
(df.isnull().sum()/df.shape[0]*100).sort_values(ascending=False)

finishedsquarefeet15                 100.00
buildingclasstypeid                  100.00
buildingclassdesc                    100.00
finishedsquarefeet13                 100.00
storydesc                             99.91
                               ...         
fips                                   0.00
bedroomcnt                             0.00
bathroomcnt                            0.00
id                                     0.00
propertylandusedesc                    0.00
Length: 69, dtype: float64

In [13]:
null_col_df = pd.DataFrame({'num_rows_missing': df.isnull().sum(),
            'percent_rows_missing': df.isnull().sum()/df.shape[0]*100})

In [14]:
null_col_df

,num_rows_missing,percent_rows_missing
parcelid,0,0.00
propertylandusetypeid,0,0.00
typeconstructiontypeid,52365,99.86
storytypeid,52394,99.91
airconditioningtypeid,38803,73.99
...,...,...
architecturalstyledesc,52371,99.87
airconditioningdesc,38803,73.99
storydesc,52394,99.91
typeconstructiondesc,52365,99.86


In [15]:
null_col_df.loc['logerror']

num_rows_missing                       0.00
percent_rows_missing                   0.00
Name: logerror, dtype: float64

In [16]:
null_col_df.iloc[5]

num_rows_missing                  52,371.00
percent_rows_missing                  99.87
Name: architecturalstyletypeid, dtype: float64

In [17]:
null_col_df.loc[['logerror']]

,num_rows_missing,percent_rows_missing
logerror,0,0.00


## Nulls by Row

In [18]:
df.isnull().sum(axis=1)

0        36
1        33
2        34
3        32
4        29
         ..
52436    34
52437    33
52438    32
52439    32
52440    34
Length: 52441, dtype: int64

In [19]:
df.isnull().sum(axis=1)/ df.shape[1] *100

0                      52.17
1                      47.83
2                      49.28
3                      46.38
4                      42.03
                ...         
52436                  49.28
52437                  47.83
52438                  46.38
52439                  46.38
52440                  49.28
Length: 52441, dtype: float64

In [20]:
null_row_df = pd.DataFrame({'num_cols_missing': df.isnull().sum(axis=1),
             'percent_cols_missing': df.isnull().sum(axis=1)/df.shape[1] *100})

In [21]:
##This function doesnt work quite right, yet
def nulls_by_rows(df):
    num_missing = df.isnull().sum(axis=1)
    percent_miss = num_missing / df.shape[1]*100
    rows = df.shape[0]
    percent_missing = num_missing/ rows *100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing' : percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [22]:
w.nulls_by_rows(df)

,num_rows_missing,percent_rows_missing
9001,48,0.09
28116,48,0.09
41468,47,0.09
13429,46,0.09
22087,45,0.09
...,...,...
12733,24,0.05
37563,24,0.05
39642,24,0.05
22117,23,0.04


In [23]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    percent_missing = num_missing/ rows *100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing' : percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [12]:
w.nulls_by_col(df)

,num_rows_missing,percent_rows_missing
finishedsquarefeet15,52441,100.00
buildingclasstypeid,52441,100.00
buildingclassdesc,52441,100.00
finishedsquarefeet13,52441,100.00
storydesc,52394,99.91
...,...,...
fips,0,0.00
bedroomcnt,0,0.00
bathroomcnt,0,0.00
id,0,0.00


In [4]:
def null_counter(df):
    new_columns = ['name','num_rows_missing', 'pct_rows_missing']
    
    new_df = pd.DataFrame(columns = new_colums)
    
    for col in list(df.columns):
        num_missing = df[col].isna().sum()
        pct_missing = num_missing / df.shape[0] #number of rows
        
        add_df = pd.DataFrame([{'name': col, 'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing}])
        
        new_df = pd.concat([new_df, add_df], axis = 0)
    
    new_df.set_index('name', inplace=True)

In [5]:
null_counter(df)

NameError: name 'new_colums' is not defined

In [25]:
##def summarize(df):
    ##'''Summarize takes a single argument and outputs various statistics on said dataframe'''

### Prepare

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [13]:
df.propertylandusedesc.unique() 

array(['Single Family Residential'], dtype=object)

#### This was done on acquire from the mySQL database

In [36]:
df[df['bathroomcnt'] == 0 ]

,parcelid,propertylandusetypeid,typeconstructiontypeid,storytypeid,airconditioningtypeid,architecturalstyletypeid,buildingclasstypeid,heatingorsystemtypeid,id,basementsqft,...,id.1,logerror,transactiondate,heatingorsystemdesc,buildingclassdesc,architecturalstyledesc,airconditioningdesc,storydesc,typeconstructiondesc,propertylandusedesc
124,14190895,261.000,NaN,NaN,NaN,NaN,NaN,NaN,2546776,NaN,...,176,-0.001,2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
2612,11696784,261.000,NaN,NaN,NaN,NaN,NaN,7.000,496818,NaN,...,3907,0.031,2017-01-19,Floor/Wall,NaN,NaN,NaN,NaN,NaN,Single Family Residential
2613,11696784,261.000,NaN,NaN,NaN,NaN,NaN,7.000,496818,NaN,...,3908,-0.125,2017-06-16,Floor/Wall,NaN,NaN,NaN,NaN,NaN,Single Family Residential
2694,12227894,261.000,NaN,NaN,NaN,NaN,NaN,NaN,1602501,NaN,...,4035,-0.025,2017-01-19,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
2899,12146026,261.000,NaN,NaN,NaN,NaN,NaN,NaN,856504,NaN,...,4333,-0.661,2017-01-20,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49855,13919043,261.000,NaN,NaN,NaN,NaN,NaN,NaN,2873456,NaN,...,73763,0.021,2017-09-05,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
50078,17292931,261.000,NaN,NaN,NaN,NaN,NaN,NaN,2117639,NaN,...,74111,0.080,2017-09-06,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential
51299,12197543,261.000,NaN,NaN,NaN,NaN,NaN,7.000,617959,NaN,...,75878,0.239,2017-09-12,Floor/Wall,NaN,NaN,NaN,NaN,NaN,Single Family Residential
52163,12399434,261.000,NaN,NaN,NaN,NaN,NaN,2.000,1759759,NaN,...,77175,0.510,2017-09-18,Central,NaN,NaN,NaN,NaN,NaN,Single Family Residential


### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

#### The input:
- A dataframe
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).


#### The output:
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
Look up the dropna documentation.
You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [14]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [15]:
#proportion missing values that have to be in rows or column
def handle_missing_values(df, prop_required_columns=0.5, prop_required_rows=0.75):
    column_threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=column_threshold)
    row_threshold = int(round(prop_required_rows * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=row_threshold)
    return df

In [16]:
w.handle_missing_values(df)

,parcelid,propertylandusetypeid,heatingorsystemtypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,14297519,261.00,NaN,1727539,3.50,4.00,NaN,3.50,"3,100.00","3,100.00",...,"1,023,282.00","2,016.00","537,569.00","11,013.72","60,590,630,072,012.00",0,0.03,2017-01-01,NaN,Single Family Residential
1,17052889,261.00,NaN,1387261,1.00,2.00,NaN,1.00,"1,465.00","1,465.00",...,"464,000.00","2,016.00","376,000.00","5,672.48","61,110,010,023,006.00",1,0.06,2017-01-01,NaN,Single Family Residential
2,14186244,261.00,NaN,11677,2.00,3.00,NaN,2.00,"1,243.00","1,243.00",...,"564,778.00","2,016.00","479,489.00","6,488.30","60,590,218,022,012.00",2,0.01,2017-01-01,NaN,Single Family Residential
3,12177905,261.00,2.00,2288172,3.00,4.00,8.00,3.00,"2,376.00","2,376.00",...,"145,143.00","2,016.00","36,225.00","1,777.51","60,373,001,001,006.00",3,-0.10,2017-01-01,Central,Single Family Residential
4,12095076,261.00,2.00,781532,3.00,4.00,9.00,3.00,"2,962.00","2,962.00",...,"773,303.00","2,016.00","496,619.00","9,516.26","60,374,608,001,014.00",6,-0.00,2017-01-01,Central,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,261.00,2.00,2274245,2.00,4.00,6.00,2.00,"1,633.00","1,633.00",...,"346,534.00","2,016.00","221,068.00","4,175.08","60,375,548,022,001.00",77607,0.00,2017-09-19,Central,Single Family Residential
52437,11000655,261.00,2.00,673515,2.00,2.00,6.00,2.00,"1,286.00","1,286.00",...,"354,621.00","2,016.00","283,704.00","4,478.43","60,371,014,003,002.00",77609,0.02,2017-09-20,Central,Single Family Residential
52438,17239384,261.00,NaN,2968375,2.00,4.00,NaN,2.00,"1,612.00","1,612.00",...,"67,205.00","2,016.00","16,522.00","1,107.48","61,110,084,022,016.00",77610,0.01,2017-09-21,NaN,Single Family Residential
52439,12773139,261.00,2.00,1843709,1.00,3.00,4.00,1.00,"1,032.00","1,032.00",...,"49,546.00","2,016.00","16,749.00",876.43,"60,374,338,022,005.00",77611,0.04,2017-09-21,Central,Single Family Residential


In [6]:
def null_dropper(df, prop_required_column, prop_required_row):
    
    prop_null_column = 1 - prop_required_column
    for col in list(df.columns):
        null_sum = df[col].isna().sum()
        null_pct = null_sum / df.shape[0]
        
        if null_pct > prop_null_column:
            df.drop(columns=col, inplace=True)
            
        row_threshold = int(prop_required_row * df.shape[1])
        
    return df

In [7]:
prop_required_row = 0.75
prop_required_row * df.shape[1]

51.75

In [8]:
df.shape

(52441, 69)

In [9]:
clean_df = null_dropper(df, prop_required_column=0.50, prop_required_row=0.75)

In [10]:
clean_df

,parcelid,propertylandusetypeid,heatingorsystemtypeid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,14297519,261.000,NaN,1727539,3.500,4.000,NaN,3.500,"3,100.000","3,100.000",...,"1,023,282.000","2,016.000","537,569.000","11,013.720","60,590,630,072,012.000",0,0.026,2017-01-01,NaN,Single Family Residential
1,17052889,261.000,NaN,1387261,1.000,2.000,NaN,1.000,"1,465.000","1,465.000",...,"464,000.000","2,016.000","376,000.000","5,672.480","61,110,010,023,006.000",1,0.056,2017-01-01,NaN,Single Family Residential
2,14186244,261.000,NaN,11677,2.000,3.000,NaN,2.000,"1,243.000","1,243.000",...,"564,778.000","2,016.000","479,489.000","6,488.300","60,590,218,022,012.000",2,0.005,2017-01-01,NaN,Single Family Residential
3,12177905,261.000,2.000,2288172,3.000,4.000,8.000,3.000,"2,376.000","2,376.000",...,"145,143.000","2,016.000","36,225.000","1,777.510","60,373,001,001,006.000",3,-0.103,2017-01-01,Central,Single Family Residential
4,12095076,261.000,2.000,781532,3.000,4.000,9.000,3.000,"2,962.000","2,962.000",...,"773,303.000","2,016.000","496,619.000","9,516.260","60,374,608,001,014.000",6,-0.001,2017-01-01,Central,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,261.000,2.000,2274245,2.000,4.000,6.000,2.000,"1,633.000","1,633.000",...,"346,534.000","2,016.000","221,068.000","4,175.080","60,375,548,022,001.000",77607,0.001,2017-09-19,Central,Single Family Residential
52437,11000655,261.000,2.000,673515,2.000,2.000,6.000,2.000,"1,286.000","1,286.000",...,"354,621.000","2,016.000","283,704.000","4,478.430","60,371,014,003,002.000",77609,0.021,2017-09-20,Central,Single Family Residential
52438,17239384,261.000,NaN,2968375,2.000,4.000,NaN,2.000,"1,612.000","1,612.000",...,"67,205.000","2,016.000","16,522.000","1,107.480","61,110,084,022,016.000",77610,0.013,2017-09-21,NaN,Single Family Residential
52439,12773139,261.000,2.000,1843709,1.000,3.000,4.000,1.000,"1,032.000","1,032.000",...,"49,546.000","2,016.000","16,749.000",876.430,"60,374,338,022,005.000",77611,0.037,2017-09-21,Central,Single Family Residential


### 3. Encapsulate your work inside of functions in a wrangle_zillow.py module.

In [45]:
#IQR
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [41]:
def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

add_upper_outlier_columns(df, k=1.5)

,parcelid,propertylandusetypeid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,fireplaceflag_outliers,structuretaxvaluedollarcnt_outliers,taxvaluedollarcnt_outliers,assessmentyear_outliers,landtaxvaluedollarcnt_outliers,taxamount_outliers,taxdelinquencyyear_outliers,censustractandblock_outliers,id.1_outliers,logerror_outliers
0,14297519,261.00,1727539,NaN,NaN,NaN,3.50,4.00,NaN,NaN,...,NaN,"35,521.25",0.00,0.00,0.00,0.00,NaN,0.00,0,0.00
1,17052889,261.00,1387261,NaN,NaN,NaN,1.00,2.00,NaN,NaN,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,"194,969,917,259.00",0,0.00
2,14186244,261.00,11677,NaN,NaN,NaN,2.00,3.00,NaN,NaN,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0.00
3,12177905,261.00,2288172,NaN,NaN,NaN,3.00,4.00,NaN,8.00,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0.00
4,12095076,261.00,781532,1.00,NaN,NaN,3.00,4.00,NaN,9.00,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52437,11000655,261.00,673515,NaN,NaN,NaN,2.00,2.00,NaN,6.00,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0.00
52438,17239384,261.00,2968375,NaN,NaN,NaN,2.00,4.00,NaN,NaN,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,"195,043,916,269.00",0,0.00
52439,12773139,261.00,1843709,1.00,NaN,NaN,1.00,3.00,NaN,4.00,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0.00
52440,12826780,261.00,1187175,NaN,NaN,NaN,2.00,3.00,NaN,6.00,...,NaN,0.00,0.00,0.00,0.00,0.00,NaN,0.00,0,0.00


In [48]:
outlier_cols = [col for col in df.columns if col.endswith('_outliers_upper')]
for col in outlier_cols:
    print(col, ': ')
    subset = df[col][df[col] >0]
    print(f'Number of Observations Above Upper Bound: {subset.count()}', '\'n')
    print(subset.describe())
    print('------', '\n')

In [49]:
outlier_cols = [col for col in df if col.endswith('_outliers')]
for col in outlier_cols:
    print('~~~\n' + col)
    data = df[col][df[col] > 0]
    print(data.describe())

~~~
parcelid_outliers
count                  20.00
mean          144,014,739.68
std            19,511,924.44
min            61,582,157.62
25%           144,902,427.12
50%           149,578,149.62
75%           149,578,800.88
max           149,629,506.62
Name: parcelid_outliers, dtype: float64
~~~
propertylandusetypeid_outliers
count                   0.00
mean                     NaN
std                      NaN
min                      NaN
25%                      NaN
50%                      NaN
75%                      NaN
max                      NaN
Name: propertylandusetypeid_outliers, dtype: float64
~~~
id_outliers
count                   0.00
mean                     NaN
std                      NaN
min                      NaN
25%                      NaN
50%                      NaN
75%                      NaN
max                      NaN
Name: id_outliers, dtype: float64
~~~
airconditioningtypeid_outliers
count               1,743.00
mean                   11.25
std        

## Mall Customers

### 1. Acquire data from the customers table in the mall_customers database.

In [3]:
import wrangle_mall as wm

In [3]:
def acquire_mall_customers():
    ''' Retrieve data from mall_customers database within codeup, selecting specific features
    If data is not present in directory, this function writes a copy as a csv file. 
    '''
    filename = "mall_customers.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the SQL query into a dataframe
        query = '''SELECT *
        FROM customers;'''
        df = pd.read_sql(query, w.get_connection('mall_customers'))

        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename, index=False)

        # Return the dataframe to the calling code
        return df

In [4]:
mall_df = wm.acquire_mall_customers()

In [5]:
mall_df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


### 2. Summarize the data (include distributions and descriptive statistics).

In [6]:
wm.summarize(mall_df)

SUMMARY REPORT


Dataframe head: 
   customer_id  gender  age  annual_income  spending_score
0            1    Male   19             15              39
1            2    Male   21             15              81
2            3  Female   20             16               6


Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
None


Dataframe Description: 
       customer_id     age  annual_income  spending_score
count      200.000 200.000        200.000         200.000
mean       100.500  38.850         60.560          50.200
std         57.879  13.969         26.265   

NameError: name 'nulls_by_col' is not defined

### 3. Detect outliers using IQR.

In [7]:
def get_upper_outliers(s, k=1.5):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, 0.75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [8]:
get_upper_outliers(mall_df.age, k=1.5)

0      0
1      0
2      0
3      0
4      0
      ..
195    0
196    0
197    0
198    0
199    0
Name: age, Length: 200, dtype: int64

In [20]:
#def add_upper_outlier_columns(df, k=1.5):
    #'''
    #Add a column with the suffix _outliers for all the numeric columns
    #in the given dataframe.
    #'''
    #for col in df.select_dtypes('number'):
    #    df[col + '_outliers_upper'] = get_upper_outliers(df[col], k)
    #return df

In [21]:
#add_upper_outlier_columns(mall_df)

,customer_id,gender,age,annual_income,spending_score,customer_id_outliers_upper,age_outliers_upper,annual_income_outliers_upper,spending_score_outliers_upper
0,1,Male,19,15,39,0,0,0.000,0
1,2,Male,21,15,81,0,0,0.000,0
2,3,Female,20,16,6,0,0,0.000,0
3,4,Female,23,16,77,0,0,0.000,0
4,5,Female,31,17,40,0,0,0.000,0
...,...,...,...,...,...,...,...,...,...
195,196,Female,35,120,79,0,0,0.000,0
196,197,Female,45,126,28,0,0,0.000,0
197,198,Male,32,126,74,0,0,0.000,0
198,199,Male,32,137,18,0,0,4.250,0


### 4. Split data into train, validate, and test.

In [7]:
mall_df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [8]:
mall_train, mall_test, mall_split = wm.train_validate_test_split(mall_df)

In [9]:
mall_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 95 to 43
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     139 non-null    int64 
 1   gender          139 non-null    object
 2   age             139 non-null    int64 
 3   annual_income   139 non-null    int64 
 4   spending_score  139 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 6.5+ KB


### 5. Encode categorical columns using a one hot encoder (pd.get_dummies).

In [10]:
pd.get_dummies(mall_train)

,customer_id,age,annual_income,spending_score,gender_Female,gender_Male
95,96,24,60,52,0,1
90,91,68,59,55,1,0
2,3,20,16,6,1,0
32,33,53,33,4,0,1
186,187,54,101,24,1,0
...,...,...,...,...,...,...
100,101,23,62,41,1,0
136,137,44,73,7,1,0
16,17,35,21,35,1,0
131,132,39,71,75,0,1


In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
#Initialize the scaler and fit/transform a couple columns in my training data
mms = MinMaxScaler()

mall_train[['age', 'annual_income']] = mms.fit_transform(mall_train[['age',
                                                                     'annual_income']])

mall_train.head()

,customer_id,gender,age,annual_income,spending_score
95,96,Male,0.115,0.369,52
90,91,Female,0.962,0.361,55
2,3,Female,0.038,0.008,6
32,33,Male,0.673,0.148,4
186,187,Female,0.692,0.705,24
